In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [15]:
#2.0版本tensorflow 中 eger（动态图） execution（执行） 是默认打开的
#数据类型
t = tf.constant([[1.,2.,3.],[4.,5.,6.]]) #常量
print(t)
print(t[:,:1])
print(t[:,1])
print(t[...,1]) #...省略号和 ：相同

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1.]
 [4.]], shape=(2, 1), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [16]:
# 运算 operations
print(t + 10) #给每个元素 + 10
print(tf.square(t)) #给每个元素平方
print(t @ tf.transpose(t)) # transpose转置 ，@ 矩阵相乘

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [21]:
# numpy and tf conversion (转变)
print(t.numpy()) #还是一个tensor 只是转变后支持numpy中的算子
print(np.square(t))
np_t = np.array([[1.,2.,3.],[4.,5.,6.]])
print(tf.constant(np_t))

[[1. 2. 3.]
 [4. 5. 6.]]
(2, 3)
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [19]:
#Scalars （0维）
t_s = tf.constant(2.718)
print(t_s.numpy())
print(t_s.shape)

2.718
()


In [24]:
#Strings 字符串
t_str = tf.constant("cafe")
print(t_str)
print(tf.strings.length(t_str))
t_str_char = "咖啡"
print(tf.strings.length(t_str_char ,unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t_str,"UTF8"))

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [26]:
#string array
t_str_arr = tf.constant(["cafe","coffee","苦咖啡"])
print(tf.strings.length(t_str_arr,unit = "UTF8_CHAR")) #返回字符串数组中每个元素的长度数组
print(tf.strings.unicode_decode(t_str_arr,"UTF8"))  #RaggedTensor

tf.Tensor([4 6 3], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [33510, 21654, 21857]]>


In [38]:
#RaggedTensor
r_tensor = tf.ragged.constant([[11,12],[21,22,23],[],[41]])

print(r_tensor)
print(r_tensor[1]) #确定下标返回是tensor
print(r_tensor[1:3]) #范围操作一定返回就是RaggedTensor

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23], []]>


In [39]:
#opertions on ragged tensor
r_tensor_2 = tf.ragged.constant([[51,52],[],[71]])
#拼接操作元素所在维度要相等
print(tf.concat([r_tensor,r_tensor_2],axis = 0))  #第一维度是行，所以将r_tensor_2的每行放在r_tensor_后面，

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [71]]>


In [40]:
print(tf.concat([r_tensor,r_tensor_2],axis = 1))  #第二维度是列，所以将r_tensor_2的每列放在r_tensor_后面

InvalidArgumentError: Input tensors have incompatible shapes.
Condition x == y did not hold.
First 1 elements of x:
[3]
First 1 elements of y:
[4]

In [41]:
r_tensor_3 = tf.ragged.constant([[13,14],[24,25,26],[],[42]])
#拼接操作元素所在维度要相等，在哪个维度上连接，那么这个维度肯定会增加。
print(tf.concat([r_tensor,r_tensor_3],axis = 1))  #第二维度是列，所以将r_tensor_2的每列放在r_tensor_后面

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 24, 25, 26], [], [41, 42]]>


In [49]:
#raggedTensor to tensor
r_tensor_4 = tf.concat([r_tensor,r_tensor_3],axis = 1)
print(r_tensor_4.to_tensor()) 

tf.Tensor(
[[11 12 13 14  0  0]
 [21 22 23 24 25 26]
 [ 0  0  0  0  0  0]
 [41 42  0  0  0  0]], shape=(4, 6), dtype=int32)


In [51]:
#raggedTensor 的 0 值都在正常值后面，所以不能用来存储稀疏矩阵，存储稀疏矩阵用sparse tensor
sp_tensor = tf.SparseTensor(indices = [[0,1],[1,0],[2,3]], #indices存在的坑，最好是排好序
                           values = [1.,2.,3.],
                           dense_shape = [3,4])
print(sp_tensor)
print(tf.sparse.to_dense(sp_tensor))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [54]:
# opertions on sparse tensor
sp_tensor2 = sp_tensor * 2.0
print(sp_tensor2)
try:
    sp_tensor3 = sp_tensor + 1
except TypeError as ex:
    print(ex)
tensor_tmp = tf.constant([[10.,20.],
                         [30.,40.],
                         [50.,60.],
                         [70.,80.]])
print(tf.sparse.sparse_dense_matmul(sp_tensor,tensor_tmp))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [56]:
sp_tensor5 = tf.SparseTensor(indices = [[0,2],[0,1],[2,3]], #indices存在的坑，最好是排好序
                           values = [1.,2.,3.],
                           dense_shape = [3,4])
print(sp_tensor5)
sp_tensor6 = tf.sparse.reorder(sp_tensor5)
print(tf.sparse.to_dense(sp_tensor6))

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [2]:
#变量 Variables
var = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
print(var)
print(var.value())
print(var.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [6]:
#使用assign函数给变量重新赋值
var.assign(2*var)
print(var.numpy(),)
var[0,1].assign(42)
print("\n")
print(var.numpy())
var[1].assign([7.,8.,9.])
print("\n")
print(var.numpy())

[[ 8. 84. 24.]
 [14. 16. 18.]]


[[ 8. 42. 24.]
 [14. 16. 18.]]


[[ 8. 42. 24.]
 [ 7.  8.  9.]]


In [7]:
#而且变量的赋值不能使用 = 赋值
try:
    var[2] = [7.,8.,9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
